For a given layer, we compare the neuron activations between two images. We will start by ranking the neurons (by index number) that have the highest activation value. We look at only the top 1000 neurons. 

We begin by loading the Inception V1 model and a dataset. We'll use the 

# Load Model

In [1]:
%%capture
import torch
import torchvision.models as models
from torchvision import transforms
import copy
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os 

# model = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True)
# model = models.vgg16(pretrained=True)
model = models.googlenet(pretrained=True)  #w/o arg, this will not pretrain it

model.eval() #set model in eval mode: https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch

# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]



---
# Load Dataset

Load kaggle.json from drive or github repo

In [ ]:
%%capture
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download iamsouravbanerjee/animal-image-dataset-90-different-animals
! unzip animal-image-dataset-90-different-animals.zip

data_dir = 'animals/animals'
f = open("name of the animals.txt", "r")
animals_list = f.readlines()
animals_list = [x.replace('\n','') for x in animals_list]

## Get Activations

To compare the activations in a layer for two images, we need to get the activations. The code below does just that.

In [ ]:
def get_activations(input_image, layer_name):
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    for name_to_check, layer in model.named_modules():
        if name_to_check == layer_name:
            break
    layer.register_forward_hook(get_activation(layer_name))
    output = model(input_batch)
    return activation.copy()  #.copy(), else will return the same actvs of model

## Sort Neurons

In [ ]:
top_neurons_dict = {}

# flatten activation tensor and sort from lowest to highest
def get_sorted_neurons(input_image, layer, filename):
    actv_layer = get_activations(input_image, layer)
    flat_layer = torch.flatten(actv_layer[layer])
    sorted, indices = torch.sort(flat_layer)
    top_neurons_dict[(filename, layer)] = indices.tolist()
    return indices.tolist()

We memoize by storing each image's layers of activations in the dictionary 'top_neurons_dict'; otherwise, we would have to calculate the layers of activations by passing the image through the model every time, which would take a lot of time when we are comparing the same activations for several pairs of images.